---
title: RToken Backtester 
description: Backtest RToken yields with Defillama historical data
show-code: False
params:
    is_custom_weights:
        label: Use custom basket weights (default is income-weighted)
        input: checkbox
        value: False    
    basket_weights:
        input: text
        label: Basket weights (comma-separated as %)
    basket_tokens:
        input: select
        label: Select collateral assets
        multi: True
        choices: ['CBETH (coinbase-wrapped-staked-eth)', 'RETH (rocket-pool)', 'SFRXETH (frax-ether)', 'SETH2 (stakewise)', 'ANKRETH (ankr)', 'USDC (flux-finance)', 'DAI (flux-finance)', 'USDT (flux-finance)', 'DAI-USDC-USDT (curve)', 'FRAX-USDC (curve)', 'FRAX-3CRV (curve)', 'LUSD-FRAXBP (convex-finance)', 'FRAX-3CRV (convex-finance)', 'FRAX-USDC (convex-finance)', 'DAI (compound)', 'USDC (compound)', 'WBTC (compound)', 'ETH (compound)', 'USDC (aave-v2)', 'DAI-USDC (uniswap-v3)', 'DAI-USDC (arrakis-finance)', 'MIM-3CRV (curve)', 'MIM-3CRV (convex-finance)', 'FRAX-USDC (uniswap-v3)', 'FPI-FRAX (frax)', 'USDC (goldfinch)', 'USDT (aave-v2)', 'USDT (compound)', 'USDC (morpho-aave)', 'DAI-USDC-USDT-SUSD (curve)', 'DAI (aave-v2)', 'WBTC (aave-v2)', 'ETH (aave-v2)', 'USDC-USDT (uniswap-v3)', 'DAI-USDC-USDT-SUSD (convex-finance)', 'LUSD-3CRV (curve)', 'TUSD-3CRV (curve)', 'TUSD-3CRV (convex-finance)', 'USDC (euler)', 'DOLA-FRAXBP (curve)', 'DOLA-FRAXBP (convex-finance)', 'USDC-EURS (curve)', 'USDC-EURS (convex-finance)', 'OUSD-3CRV (curve)', 'OUSD-3CRV (convex-finance)', 'USDC (compound-v3)', 'DAI-USDC (uniswap-v2)', 'ALUSD-FRAXBP (curve)', 'ALUSD-3CRV (curve)', 'ALUSD-3CRV (convex-finance)', 'DAI (morpho-compound)', 'USDT (euler)', 'DAI-USDC-USDT (convex-finance)', 'NUSDC (notional)', 'BEAN-3CRV (curve)', 'USDC (morpho-compound)', 'DAI (uwu-lend)', 'BUSD-USDC (uniswap-v3)', 'ADAI-AUSDC-AUSDT (curve)', 'CDAI-CUSDC (curve)', 'DAI-FRAX (uniswap-v3)', 'stETH (lido)', 'ETH-STETH (curve)', 'ETH-STETH (convex-finance)', 'WSTETH-WETH (balancer)', 'WSTETH-WETH (aura)', 'USDT-WBTC-WETH (curve)', 'USDT-WBTC-WETH (convex)', 'USDC (flux)', 'USDC (morpho)', 'USDC (compound-v3']
    reference_token:
        input: select
        label: Select reference asset
        choices: ['CBETH (coinbase-wrapped-staked-eth)', 'RETH (rocket-pool)', 'SFRXETH (frax-ether)', 'SETH2 (stakewise)', 'ANKRETH (ankr)', 'USDC (flux-finance)', 'DAI (flux-finance)', 'USDT (flux-finance)', 'DAI-USDC-USDT (curve)', 'FRAX-USDC (curve)', 'FRAX-3CRV (curve)', 'LUSD-FRAXBP (convex-finance)', 'FRAX-3CRV (convex-finance)', 'FRAX-USDC (convex-finance)', 'DAI (compound)', 'USDC (compound)', 'WBTC (compound)', 'ETH (compound)', 'USDC (aave-v2)', 'DAI-USDC (uniswap-v3)', 'DAI-USDC (arrakis-finance)', 'MIM-3CRV (curve)', 'MIM-3CRV (convex-finance)', 'FRAX-USDC (uniswap-v3)', 'FPI-FRAX (frax)', 'USDC (goldfinch)', 'USDT (aave-v2)', 'USDT (compound)', 'USDC (morpho-aave)', 'DAI-USDC-USDT-SUSD (curve)', 'DAI (aave-v2)', 'WBTC (aave-v2)', 'ETH (aave-v2)', 'USDC-USDT (uniswap-v3)', 'DAI-USDC-USDT-SUSD (convex-finance)', 'LUSD-3CRV (curve)', 'TUSD-3CRV (curve)', 'TUSD-3CRV (convex-finance)', 'USDC (euler)', 'DOLA-FRAXBP (curve)', 'DOLA-FRAXBP (convex-finance)', 'USDC-EURS (curve)', 'USDC-EURS (convex-finance)', 'OUSD-3CRV (curve)', 'OUSD-3CRV (convex-finance)', 'USDC (compound-v3)', 'DAI-USDC (uniswap-v2)', 'ALUSD-FRAXBP (curve)', 'ALUSD-3CRV (curve)', 'ALUSD-3CRV (convex-finance)', 'DAI (morpho-compound)', 'USDT (euler)', 'DAI-USDC-USDT (convex-finance)', 'NUSDC (notional)', 'BEAN-3CRV (curve)', 'USDC (morpho-compound)', 'DAI (uwu-lend)', 'BUSD-USDC (uniswap-v3)', 'ADAI-AUSDC-AUSDT (curve)', 'CDAI-CUSDC (curve)', 'DAI-FRAX (uniswap-v3)', 'stETH (lido)', 'ETH-STETH (curve)', 'ETH-STETH (convex-finance)', 'WSTETH-WETH (balancer)', 'WSTETH-WETH (aura)', 'USDT-WBTC-WETH (curve)', 'USDT-WBTC-WETH (convex)', 'USDC (flux)', 'USDC (morpho)', 'USDC (compound-v3']
---

In [3]:
import pandas as pd
import numpy as np
import requests as req
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
pool_dict = {
    'CBETH (coinbase-wrapped-staked-eth)': '0f45d730-b279-4629-8e11-ccb5cc3038b4',
    'RETH (rocket-pool)': 'd4b3c522-6127-4b89-bedf-83641cdcd2eb',
    'SFRXETH (frax-ether)': '77020688-e1f9-443c-9388-e51ace15cc32',
    'SETH2 (stakewise)': '66958f46-1d06-4f83-9fab-bbec354049d8',
    'ANKRETH (ankr)': 'e201dbed-63fa-48e2-bfa2-f56e730167d2',
    'USDC (flux-finance)': 'fa4d7ee4-0001-4133-9e8d-cf7d5d194a91',
    'DAI (flux-finance)': 'ed227286-abb0-4a34-ada5-39f7ebd81afb',
    'USDT (flux-finance)': '6600934f-6323-447d-8a7d-67fbede8529d',
    'DAI-USDC-USDT (curve)': '25171c4c-1877-449a-9f88-45a9f153ee31',
    'FRAX-USDC (curve)': '3f6aa14f-eb0c-4738-bf74-8bc666f7d2b1',
    'FRAX-3CRV (curve)': '8d989d6c-99ba-447e-8f3d-3aca98e993fd',
    'LUSD-FRAXBP (convex-finance)' : 'c3b238b8-e1b8-43a5-8079-99a73f3dc981',
    'FRAX-3CRV (convex-finance)': '844a8a29-4653-42e6-b675-a6b43fac3678',
    'FRAX-USDC (convex-finance)': 'bd072651-d99c-4154-aeae-51f12109c054',
    'DAI (compound)': 'cc110152-36c2-4e10-9c12-c5b4eb662143',
    'USDC (compound)': 'cefa9bb8-c230-459a-a855-3b94e96acd8c',
    'WBTC (compound)': '1d876729-4445-4623-8b6b-c5290db5d100',
    'ETH (compound)': '1e5da7c6-59bb-49bd-9f97-4f4fceeffad4',
    'USDC (aave-v2)': 'a349fea4-d780-4e16-973e-70ca9b606db2',
    'DAI-USDC (uniswap-v3)': '26298be5-691a-42e6-817b-07f72e70cd4f',
    'DAI-USDC (arrakis-finance)': '2431aeec-d0b9-4e62-ab5e-ff8718ad45fc',
    'MIM-3CRV (curve)': 'fb6676ba-0a45-44c8-86f6-63956956ebe1',
    'MIM-3CRV (convex-finance)': '8a20c472-142c-4442-b724-40f2183c073e',
    'FRAX-USDC (uniswap-v3)': 'd371fc69-9a77-4336-a129-ec79c9548266',
    'FPI-FRAX (frax)': 'a61791ea-2886-4428-991c-7181d26a6062',
    'USDC (goldfinch)': 'c89af47f-032a-4c59-bdc9-17a7f1514a3b',
    'USDT (aave-v2)': '60d657c9-5f63-4771-a85b-2cf8d507ec00',
    'USDT (compound)': '57647093-2868-4e65-97ab-9cae8ec74e7d',
    'USDC (morpho-aave)': '325ad2d6-70b1-48d7-a557-c2c99a036f87',
    'DAI-USDC-USDT-SUSD (curve)': '985b3a44-495f-4250-bde3-7b2299036e3e',
    'DAI (aave-v2)': '405d8dad-5c99-4c91-90d3-82813ade1ff1',
    'WBTC (aave-v2)': '40e0d8b7-5eba-4d6c-a3c2-9ac375bcd4bf',
    'ETH (aave-v2)': 'f2726d05-1f8d-4b9c-80e3-43d03d85d117',
    'USDC-USDT (uniswap-v3)': 'e737d721-f45c-40f0-9793-9f56261862b9',
    'DAI-USDC-USDT-SUSD (convex-finance)': '45f74ed5-5832-4d6d-8f2c-2929b6ac5e9c',
    'LUSD-3CRV (curve)': '836e7def-cb99-46f6-8e22-1056a81e4040',
    'TUSD-3CRV (curve)': '0023d0df-b295-4bca-9e4f-12053445460f',
    'TUSD-3CRV (convex-finance)': 'fde2777a-952a-49bd-b0be-b28bd35b7718',
    'USDC (euler)': '61b7623c-9ac2-4a73-a748-8db0b1c8c5bc',
    'DOLA-FRAXBP (curve)': '3a745337-3f04-409d-9627-8391e3c7494d',
    'DOLA-FRAXBP (convex-finance)': 'd05cb04d-f1e5-451d-95a2-6a3a9da001ad',
    'USDC-EURS (curve)': '4dbfda50-4474-4ef1-9f59-b5cf5ad7413c',
    'USDC-EURS (convex-finance)': 'efa8dc4f-03df-4633-95bf-db188292d9a3',
    'OUSD-3CRV (curve)': '4fcac035-f39c-4238-bc87-4c4cfe0b3092',
    'OUSD-3CRV (convex-finance)': '9e038028-8f56-449f-8410-64683f0c086b',
    'USDC (compound-v3)': '7da72d09-56ca-4ec5-a45f-59114353e487',
    'DAI-USDC (uniswap-v2)': 'a86ee795-54d9-4812-9148-b312967cefe5',
    'ALUSD-FRAXBP (curve)': '71a56b67-3123-4820-a30f-fac238de441c',
    'ALUSD-3CRV (curve)': 'fa1ea651-4d34-4672-837f-9dcd043319b9',
    'ALUSD-3CRV (convex-finance)': 'f11aa021-e4c1-49f9-8004-9cf5625dedae',
    'DAI (morpho-compound)': '1e6a12f5-687c-4ce6-8416-af38f17954ea',
    'USDT (euler)': '618c5e6c-78f3-4e20-9182-b16e5ef20292',
    'DAI-USDC-USDT (convex-finance)': '7394f1bc-840a-4ff0-9e87-5e0ef932943a',
    'NUSDC (notional)': '7131b94d-a801-446c-9968-8fa7f68b3b87',
    'BEAN-3CRV (curve)': '3e2dbffc-b5b9-4d93-8236-46969f83d64b',
    'USDC (morpho-compound)': 'b718e8ee-c749-4152-9f3d-e3f9ff2da564',
    'DAI (uwu-lend)': '45a43ff6-9320-4e40-9812-ba91d6f39307',
    'BUSD-USDC (uniswap-v3)': 'daef836d-0e55-41ba-81fc-335234eef50e',
    'ADAI-AUSDC-AUSDT (curve)': 'e3f861c0-c2b9-4b50-b799-a036a3f11da1',
    'CDAI-CUSDC (curve)': '8ec9e3e2-1eb5-4c47-82de-edcc825ca4cc',
    'DAI-FRAX (uniswap-v3)': 'd6e3dd14-0495-4aec-b226-ae98215dcd38',
    'stETH (lido)' : '747c1d2a-c668-4682-b9f9-296708a3dd90',
    'ETH-STETH (curve)': '57d30b9c-fc66-4ac2-b666-69ad5f410cce',
    'ETH-STETH (convex-finance)': '5ce23e7e-3800-4c9c-ad30-6db3db0515a1',
    'WSTETH-WETH (balancer)': '4cc5df76-f81d-49fe-9e1e-4caa6a8dad0b',
    'WSTETH-WETH (aura)': '6429fd3b-af61-4e15-a4b4-34819fb8f4ea',
    'USDT-WBTC-WETH (curve)': '9e4dbe67-f5fc-4428-8e6f-93d464f40a0d',
    'USDT-WBTC-WETH (convex)': 'ad3d7253-fb8f-402f-a6f8-821bc0a055cb',
    'USDC (flux)': 'fa4d7ee4-0001-4133-9e8d-cf7d5d194a91',
    'USDC (morpho)': '325ad2d6-70b1-48d7-a557-c2c99a036f87',
    'USDC (compound-v3': '7da72d09-56ca-4ec5-a45f-59114353e487',
}


In [ ]:
# Raw pool data in dictionaries 

pool_dict_raw = dict(pd.DataFrame(req.get('https://yields.llama.fi/pools').json())['data'])

# Dataframe of raw data
df_raw = pd.DataFrame.from_dict(pool_dict_raw, orient='index')
df_raw = df_raw.sort_values(by ='tvlUsd', ascending = False)
eth_pools = df_raw[df_raw.chain == 'Ethereum']

In [ ]:
# Set Parameters for Basket

min_apy = 0
is_stablecoin = True
min_tvl = 80000000
top_x = 15
sort_by = 'apy' # or tvlUsd or apy

In [ ]:
basket_tokens = []
reference_token = "USDC (Compound)"
basket_weights = ""
is_custom_weights = False

In [ ]:
using_custom_weights = is_custom_weights and len(basket_tokens) == len(basket_weights.split(','))

benchmark = pool_dict[reference_token]
bespoke = [pool_dict[token] for token in basket_tokens]
weights = [float(weight) if using_custom_weights else 0 for weight in basket_weights.split(',')]
normalized_weights = [weight/sum(weights) for weight in weights if using_custom_weights]

pool_symbols = eth_pools[eth_pools.pool.isin(bespoke)][['project','symbol','pool', 'tvlUsd','apy']]
benchmark_symbols = eth_pools[eth_pools.pool == benchmark][['project','symbol','pool', 'tvlUsd','apy']]

In [ ]:
# Creates a table with pool symbol and pool ids to be later used for joining 

def symbol_ops(df):
    # calc for income
    df['income'] = df.tvlUsd * df.apy

    # calc for income weighted basket based on current TVL and APY 
    df['income_basket_weight'] = round(df.income / sum(df.income),2)
        
    # concat the pool symbol to show pool and protocol
    df['pool (protocol)'] = df.symbol + " (" + df.project + ")"

    #drop columns that we no longer need
    df= df.drop(['tvlUsd','apy','income'], axis=1)
    
    return df
    

# Establishes a list of dataframes of historical data for each of the pools in the desired basket
# Input is the dataframe returned by symbol_ops() 

def create_historical_dfs(df):
    basket_pool_ids = list(df.pool)
    
    pool_dfs = []
    for i in basket_pool_ids:
        data = pd.DataFrame()
        data = pd.DataFrame(req.get('https://yields.llama.fi/chart/'+ str(i)).json()['data'])
        data = data[['timestamp','tvlUsd','apy']]
        data['timestamp'] = [i[0:10] for i in list(data.timestamp)]
        data['timestamp'] = pd.to_datetime(data.timestamp)
        data['pool_id'] = str(i)

        # joins pool symbol onto dataframe using pool id
        data = pd.merge(data, df, how='left', 
             left_on= 'pool_id', right_on='pool' )[['timestamp','tvlUsd','apy','income_basket_weight','pool (protocol)']]

        # adds column for portfolio yield contribution (apy * income_basket_weight)
        data['portfolio_yield_contribution'] = data.iloc[:,2] * data.iloc[:,3]

        # renames columns to reflect pool symbol and drops pool symbol column
        pool_protocol_string = data['pool (protocol)'][0]
        data.columns = ['timestamp',  
                          ('tvlUsd | ' + pool_protocol_string), 
                           ('apy | ' + pool_protocol_string),
                        'income_basket_weight | ' + pool_protocol_string,
                        'pool (protocol)',
                       'portfolio_yield_contribution | ' + pool_protocol_string]
        data = data.drop('pool (protocol)', axis=1)

        # creates and calculates column for income (apy column has more decimal points than shown, results in slight math discrepency on income)
        #income_column_string = 'income | ' + pool_protocol_string
        #data[income_column_string] = data.iloc[:, 1] * data.iloc[:, 2] / 100

        pool_dfs.append(data)
        
    return pool_dfs


#Checks to see which pool within the basket has the least amount of historical data
# Input is the list of dfs returned by create_historical_dfs()
def count_days_by_pool(pool_dfs_list):
    for i in pool_dfs_list:
        print( (list(i)[1][8:], ' : ', len(i)))


        
# Creates master dataframe with historical data froma all pools, using the pool that has the least hisory as base, 
# and caluculates the total_portfolio_yield for each day

# Input is the list of dfs returned by create_historical_dfs()

def calc_master_df(pool_dfs_list):
    from functools import reduce
    df_master = reduce(lambda x, y: pd.merge(x, y, on = 'timestamp', how ='left'), sorted(pool_dfs_list, key=len))
    
    yield_contribution_cols = [col for col in df_master.columns if 'portfolio_yield_contribution |' in col]

    # caluculates the total_portfolio_yield for each day
    df_master['total_portfolio_yield'] = df_master[yield_contribution_cols].sum(axis=1)
    
    return df_master

In [ ]:
pool_symbols = symbol_ops(pool_symbols)
if(using_custom_weights):
    pool_symbols['income_basket_weight'] = normalized_weights
    
benchmark_symbols = symbol_ops(benchmark_symbols)


## RToken Basket Composition

In [ ]:
pool_symbols

## Benchmark

In [ ]:
benchmark_symbols

In [ ]:
pool_dfs_basket = create_historical_dfs(pool_symbols)
pool_dfs_benchmark = create_historical_dfs(benchmark_symbols)

In [ ]:
df = calc_master_df(pool_dfs_basket)

df_benchmark = calc_master_df(pool_dfs_benchmark)

# makes basket and benchmark dataframes the same length
num_days_of_data = min(len(df),len(df_benchmark))

df = df.tail(num_days_of_data)
df_benchmark = df_benchmark.tail(num_days_of_data)

## Backtest Results

In [ ]:
_ = sns.lineplot(x = "timestamp", y = "total_portfolio_yield",
             data = df)
_ = sns.lineplot(x = "timestamp", y = "total_portfolio_yield",
                data = df_benchmark, color="orange")
_ = plt.title('Benchmark Yield Over Time')
_ = plt.ylabel('APY')
_ = plt.xlabel('Date')
_ = plt.legend(['RToken Basket Yield','Benchmark'])
_ = plt.xticks(rotation = 45)
